In [100]:
import os 
import torch
import torch.nn as nn 
w_ori = torch.load("FSSD_VGG_VOC_epoches_224.pth") # add BN and l1 for FSSD 
bn = []
percent = 0.5 # prune therefold
for k,v in w_ori.items():
    print (k,'\t',v.shape)
    if k.endswith("bn.weight"):
        bn += list(v.data.cpu().abs().numpy())
total = len(bn)
thre_num = int(total*percent)
bn.sort()
thre = bn[thre_num]
thre = torch.tensor(float(thre)).cuda()
mask_cfg = []
cfg = []
for k,v in w_ori.items():
    if k.endswith("bn.weight"):
        mask = v.data.abs().gt(thre)
        mask_cfg.append(mask)
        cfg.append(torch.sum(mask).cpu().item())
print (cfg)

base.0.conv.weight 	 torch.Size([64, 3, 3, 3])
base.0.bn.weight 	 torch.Size([64])
base.0.bn.bias 	 torch.Size([64])
base.0.bn.running_mean 	 torch.Size([64])
base.0.bn.running_var 	 torch.Size([64])
base.0.bn.num_batches_tracked 	 torch.Size([])
base.2.conv.weight 	 torch.Size([64, 64, 3, 3])
base.2.bn.weight 	 torch.Size([64])
base.2.bn.bias 	 torch.Size([64])
base.2.bn.running_mean 	 torch.Size([64])
base.2.bn.running_var 	 torch.Size([64])
base.2.bn.num_batches_tracked 	 torch.Size([])
base.5.conv.weight 	 torch.Size([128, 64, 3, 3])
base.5.bn.weight 	 torch.Size([128])
base.5.bn.bias 	 torch.Size([128])
base.5.bn.running_mean 	 torch.Size([128])
base.5.bn.running_var 	 torch.Size([128])
base.5.bn.num_batches_tracked 	 torch.Size([])
base.7.conv.weight 	 torch.Size([128, 128, 3, 3])
base.7.bn.weight 	 torch.Size([128])
base.7.bn.bias 	 torch.Size([128])
base.7.bn.running_mean 	 torch.Size([128])
base.7.bn.running_var 	 torch.Size([128])
base.7.bn.num_batches_tracked 	 torch.Size([]

In [112]:
import numpy as np
start_cfg = torch.ones(3)
layer_id = 0
end_cfg = mask_cfg[layer_id]
ft_start = ["base.21.conv.weight","base.33.conv.weight","extras.3.conv.weight"]
ft = []
py0_ext = []
from collections import OrderedDict
new_weight = OrderedDict()
for k,v in w_ori.items():
    if 'conv' in k:
        if not k.startswith("ft_module") and k != "pyramid_ext.0.conv.weight":
            idx0 = np.squeeze(np.argwhere(start_cfg.cpu().numpy()))
            idx1 = np.squeeze(np.argwhere(end_cfg.cpu().numpy()))
            w1 = v.data[:,idx0.tolist(),:,:]
            new_weight[k] = w1[idx1.tolist(),:,:,:]
            if k in ft_start:
                ft.append(idx1.tolist())
        elif k != "pyramid_ext.0.conv.weight":
            idx0 = ft[0]
            idx1 = np.squeeze(np.argwhere(end_cfg.cpu().numpy()))
            w1 = v.data[:,idx0,:,:]
            new_weight[k] = w1[idx1.tolist(),:,:,:]
            py0_ext.append(idx1.tolist())
            ft.pop(0)
        else:
            idx0 = py0_ext[0] + [256+i for i in py0_ext[1]] + [512+i for i in py0_ext[2]]
            idx1 = np.squeeze(np.argwhere(end_cfg.cpu().numpy())) 
            w1 = v.data[:,idx0,:,:]
            new_weight[k] = w1[idx1.tolist(),:,:,:]
    elif 'bn' in k:
        if 'num_batches_tracked' not in k:
            idx1 = np.squeeze(np.argwhere(end_cfg.cpu().numpy()))
            new_weight[k] = v.data[idx1.tolist()]
        else:
            new_weight[k] = v
            start_cfg = end_cfg
            layer_id += 1
            if layer_id < len(mask_cfg):
                end_cfg = mask_cfg[layer_id]
    else:
        new_weight[k] = v
torch.save(new_weight,'prune.pth')

1	 base.0.conv.weight
1	 base.2.conv.weight
1	 base.5.conv.weight
1	 base.7.conv.weight
1	 base.10.conv.weight
1	 base.12.conv.weight
1	 base.14.conv.weight
1	 base.17.conv.weight
1	 base.19.conv.weight
1	 base.21.conv.weight
1	 base.24.conv.weight
1	 base.26.conv.weight
1	 base.28.conv.weight
1	 base.31.conv.weight
1	 base.33.conv.weight
1	 extras.0.conv.weight
1	 extras.1.conv.weight
1	 extras.2.conv.weight
1	 extras.3.conv.weight
179
1	 pyramid_ext.1.conv.weight
1	 pyramid_ext.2.conv.weight
1	 pyramid_ext.3.conv.weight
1	 pyramid_ext.4.conv.weight
1	 pyramid_ext.5.conv.weight


In [113]:
w0 = torch.load("prune.pth")
for k,v in w0.items():
    print (k,'\t',v.shape)
 

base.0.conv.weight 	 torch.Size([38, 3, 3, 3])
base.0.bn.weight 	 torch.Size([38])
base.0.bn.bias 	 torch.Size([38])
base.0.bn.running_mean 	 torch.Size([38])
base.0.bn.running_var 	 torch.Size([38])
base.0.bn.num_batches_tracked 	 torch.Size([])
base.2.conv.weight 	 torch.Size([58, 38, 3, 3])
base.2.bn.weight 	 torch.Size([58])
base.2.bn.bias 	 torch.Size([58])
base.2.bn.running_mean 	 torch.Size([58])
base.2.bn.running_var 	 torch.Size([58])
base.2.bn.num_batches_tracked 	 torch.Size([])
base.5.conv.weight 	 torch.Size([101, 58, 3, 3])
base.5.bn.weight 	 torch.Size([101])
base.5.bn.bias 	 torch.Size([101])
base.5.bn.running_mean 	 torch.Size([101])
base.5.bn.running_var 	 torch.Size([101])
base.5.bn.num_batches_tracked 	 torch.Size([])
base.7.conv.weight 	 torch.Size([121, 101, 3, 3])
base.7.bn.weight 	 torch.Size([121])
base.7.bn.bias 	 torch.Size([121])
base.7.bn.running_mean 	 torch.Size([121])
base.7.bn.running_var 	 torch.Size([121])
base.7.bn.num_batches_tracked 	 torch.Size([]

TypeError: unsupported operand type(s) for +: 'int' and 'list'